In [1]:
!pip install tensorflow-addons

In [2]:
!pip install wandb -qqq
!nvidia-smi

Fri Oct  8 14:42:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    46W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
WANDB = True
ver = 'v28'
API_KEY = '4dad92fd17d935955886d8b751df8aa51f7a8523'
TEST_PREDS_DATA_PATH = '/content/drive/MyDrive/kaggle/pressure prediction/dl_sub_v27.csv'

In [4]:

import wandb
wandb.login(key=API_KEY)


wandb: Currently logged in as: pressureprediction (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
import pandas as pd
import numpy as np
from IPython.display import display
from sklearn.model_selection import train_test_split, GroupKFold, KFold
from sklearn.metrics import mean_absolute_error as mae
from tensorflow import keras
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.preprocessing import normalize
import gc
from time import time
import os

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, Input, BatchNormalization
from tensorflow.keras.layers import *
from tensorflow.keras import callbacks, optimizers
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from sklearn.preprocessing import RobustScaler, LabelEncoder

In [7]:
DATA_PATH = '/content/drive/MyDrive/kaggle/pressure prediction'

submission = pd.read_csv(DATA_PATH + os.sep + 'sample_submission.csv')
train = pd.read_csv(DATA_PATH + os.sep + 'train.csv')
test = pd.read_csv(DATA_PATH + os.sep + 'test.csv')
test['pressure'] = pd.read_csv(TEST_PREDS_DATA_PATH)['pressure']

In [8]:
fltr_out = train['u_out'] == 0
target = train['pressure']

In [9]:
train['u_in_2'] = train['u_in'].copy()
test['u_in_2'] = test['u_in'].copy()

In [10]:
fltr = train['u_out'] == 0
train.loc[fltr, 'u_in_2'] = 0
test.loc[fltr, 'u_in_2'] = 0

In [11]:
oof_df = train[['id']].copy()

In [12]:
SAMPLE_TRAIN = False
SAMPLE_TEST = False

In [13]:
np.random.seed(10)

if SAMPLE_TRAIN:
    sampled = np.random.choice(train['breath_id'].unique(), size=5000, replace=False)
    print(train.shape)
    train = train[train['breath_id'].isin(sampled)].reset_index(drop=True)
    print(train.shape)

if SAMPLE_TEST:
    sampled = np.random.choice(test['breath_id'].unique(), size=5000, replace=False)
    test = test[test['breath_id'].isin(sampled)].reset_index(drop=True)

In [14]:
np.random.seed(10)
EVAL = False
TOTAL_BREATHS = train['breath_id'].nunique()
TOTAL_TRAIN_BREATHS = int(TOTAL_BREATHS * 0.8)
if EVAL:
    sampled = np.random.choice(train['breath_id'].unique(), size=TOTAL_BREATHS, replace=False)
    sample_train = sampled[:TOTAL_TRAIN_BREATHS]
    sample_test = sampled[TOTAL_TRAIN_BREATHS:]
    print(len(sample_test))
    print(train.shape)
    print(test.shape)
    test = train[train['breath_id'].isin(sample_test)].reset_index(drop=True)
    train = train[train['breath_id'].isin(sample_train)].reset_index(drop=True)
    
    print(train.shape)
    print(test.shape)

    test_preds = test['pressure']
    del test['pressure']

In [15]:
train['bidc'] = train.groupby('breath_id').cumcount()
test['bidc'] = test.groupby('breath_id').cumcount()

In [16]:
train = train[train['bidc'] <= 38].reset_index(drop=True)
test = test[test['bidc'] <= 38].reset_index(drop=True)

In [17]:
sample_oof_df = train[['id']].copy()
sample_preds_df = test[['id']].copy()

In [18]:
for df_tmp in [train, test]:
    df_tmp['R_cat'] = df_tmp['R'].map({20: 1, 50: 2, 5: 0})
    df_tmp['C_cat'] = df_tmp['C'].map({20: 1, 50: 2, 10: 0})
    df_tmp['RC_cat'] = df_tmp['R'].astype('str')+'-' + df_tmp['C'].astype('str')
    

mapper = pd.Series(index=train['RC_cat'].unique(), data=np.arange(train['RC_cat'].nunique()))
for df_tmp in [train, test]:
    df_tmp['RC_cat'] = df_tmp['RC_cat'].map(mapper)

cat_cols = ['R_cat', 'C_cat', 'RC_cat']
cat_cols_unq_dct = {}
for c in cat_cols:
    cat_cols_unq_dct[c] = train[c].nunique()

In [19]:
c = 'u_in'
LAG_WINDOW_RANGE = range(3)
train = train.assign(**{f'{c}_t-{t}': train.groupby('breath_id')[c].shift(t) for t in LAG_WINDOW_RANGE})
test = test.assign(**{f'{c}_t-{t}': test.groupby('breath_id')[c].shift(t) for t in LAG_WINDOW_RANGE})
use_fts = [f'{c}_t-{t}' for t in LAG_WINDOW_RANGE]

In [20]:
for df_tmp in [train, test]:
    df_tmp['u_in_lag_1'] = df_tmp.groupby('breath_id')['u_in'].shift(1).fillna(0)
    df_tmp['u_in_lag_2'] = df_tmp.groupby('breath_id')['u_in'].shift(2).fillna(0)
    df_tmp['u_in_lag_3'] = df_tmp.groupby('breath_id')['u_in'].shift(3).fillna(0)
    df_tmp['u_in_lag_4'] = df_tmp.groupby('breath_id')['u_in'].shift(4).fillna(0)

    df_tmp['u_in_cumsum'] = df_tmp.groupby('breath_id')['u_in'].cumsum()
    df_tmp['u_in_cumsum_lag_1'] = df_tmp.groupby('breath_id')['u_in_cumsum'].shift(1).fillna(0)
    df_tmp['u_in_cumsum_lag_2'] = df_tmp.groupby('breath_id')['u_in_cumsum'].shift(2).fillna(0)
    df_tmp['u_in_cumsum_lag_3'] = df_tmp.groupby('breath_id')['u_in_cumsum'].shift(3).fillna(0)
    df_tmp['u_in_cumsum_lag_4'] = df_tmp.groupby('breath_id')['u_in_cumsum'].shift(4).fillna(0)

    df_tmp['u_in_cummean'] = df_tmp['u_in_cumsum']/(df_tmp.groupby('breath_id')['u_in'].cumcount() + 1)
    df_tmp['u_in_cummax'] = df_tmp.groupby('breath_id')['u_in'].cummax()
    df_tmp['next_u_in'] = df_tmp.groupby('breath_id')['u_in'].shift(-1).fillna(0)

    df_tmp['roll_u_in_max'] = df_tmp[use_fts].max(axis=1).fillna(0)
    df_tmp['roll_u_in_min'] = df_tmp[use_fts].min(axis=1).fillna(0)

    # df_tmp['time_lag_1'] = df_tmp.groupby('breath_id')['time_step'].shift(1).fillna(0)
    # df_tmp['time_lag_2'] = df_tmp.groupby('breath_id')['time_step'].shift(2).fillna(0)

    df_tmp.drop(use_fts, axis=1, inplace=True)

    df_tmp['area'] = df_tmp['time_step'] * df_tmp['u_in']
    df_tmp['area'] = df_tmp.groupby('breath_id')['area'].cumsum()
    df_tmp['area_lag_1'] = df_tmp.groupby('breath_id')['area'].shift(1).fillna(0)
    df_tmp['area_lag_2'] = df_tmp.groupby('breath_id')['area'].shift(2).fillna(0)
    df_tmp['area_lead_1'] = df_tmp.groupby('breath_id')['area'].shift(-1).fillna(0)
    df_tmp['area_lead_2'] = df_tmp.groupby('breath_id')['area'].shift(-2).fillna(0)
    df_tmp['area_diff_lag_1'] = df_tmp['area'] - df_tmp['area_lag_1']
    df_tmp['area_diff_lead_1'] = df_tmp['area'] - df_tmp['area_lead_1']
    
    df_tmp['u_out_lag_1'] = df_tmp.groupby('breath_id')['u_out'].shift(1).fillna(0)
    df_tmp['u_out_lag_2'] = df_tmp.groupby('breath_id')['u_out'].shift(2).fillna(0)

    df_tmp['time_step*u_out']= df_tmp['time_step']*df_tmp['u_out']


    df_tmp['R+C'] = df_tmp['R'] + df_tmp['C']
    df_tmp['R/C'] = df_tmp['R'] / df_tmp['C']
    df_tmp['u_in/C'] = df_tmp['u_in'] / df_tmp['C']
    df_tmp['u_in/R'] = df_tmp['u_in'] / df_tmp['R']
    df_tmp['u_in_cumsum/C'] = df_tmp['u_in_cumsum'] / df_tmp['C']
    df_tmp['u_in_cumsum/R'] = df_tmp['u_in_cumsum'] / df_tmp['R']
    df_tmp['area*R/C'] = df_tmp['area'] * df_tmp['R/C']
    df_tmp['u_in_cumsum*R/C'] = df_tmp['u_in_cumsum'] * df_tmp['R/C']
    df_tmp['u_in_cumsum*R/C_lag_1'] = df_tmp.groupby('breath_id')['u_in_cumsum*R/C'].shift(1).fillna(0)


    df_tmp['timestep_diff'] = (df_tmp['time_step'] - df_tmp.groupby('breath_id')['time_step'].shift(1)).fillna(0)
    df_tmp['u_in_diff'] = (df_tmp['u_in'] - df_tmp.groupby('breath_id')['u_in'].shift(1)).fillna(0)
    df_tmp['u_in_pct_change'] = (df_tmp['u_in_diff']/(df_tmp['u_in_lag_1'] + 1e-4)).fillna(0)
    df_tmp['u_in_diff_next'] = (df_tmp['u_in'] - df_tmp.groupby('breath_id')['u_in'].shift(-1)).fillna(0)
    df_tmp['u_in_log'] = np.log1p(df_tmp['u_in'])
    df_tmp['u_in_cumsum_log'] = np.log1p(df_tmp['u_in_cumsum'])

In [21]:
num_cols = [c for c in train.columns if c not in ['id', 'pressure', 'breath_id', 'u_out'] + cat_cols]

In [22]:
RS = RobustScaler()
train[num_cols] = RS.fit_transform(train[num_cols])
test[num_cols] = RS.transform(test[num_cols])

In [23]:
class VentilatorDataset:
    def __init__(self, df, cat_cols, num_cols):
        if "pressure" not in df.columns:
            df['pressure'] = 0

        self.pressures = df[['pressure']].to_numpy().reshape(-1, df['bidc'].nunique())
        self.u_outs = df[['u_out']].to_numpy().reshape(-1, df['bidc'].nunique())
        self.breath_ids = df[['breath_id']].to_numpy().reshape(-1, df['bidc'].nunique())
        self.breath_ids_ser = df['breath_id'].drop_duplicates()
        self.inputs = df[num_cols + cat_cols].values.reshape(-1, df['bidc'].nunique(), len(num_cols + cat_cols))

In [24]:
test_breath_ids = test['breath_id'].unique()
breath_ids_a = test_breath_ids[:len(test_breath_ids)//2]

fltr = test['breath_id'].isin(breath_ids_a)
test_a = test[fltr].reset_index(drop=True)
test_b = test[~fltr].reset_index(drop=True)

print(test_a.shape, test_b.shape)

(980850, 52) (980850, 52)


In [25]:
%%time
test_data_a = VentilatorDataset(test_a, cat_cols, num_cols)
test_data_b = VentilatorDataset(test_b, cat_cols, num_cols)
train_data = VentilatorDataset(train, cat_cols, num_cols)
_ = gc.collect()

CPU times: user 1.59 s, sys: 16.3 ms, total: 1.61 s
Wall time: 1.6 s


In [26]:
train_data.inputs.shape, train_data.pressures.shape, train_data.u_outs.shape

((75450, 39, 48), (75450, 39), (75450, 39))

In [27]:
test_data_a.inputs.shape, test_data_a.pressures.shape, test_data_a.u_outs.shape

((25150, 39, 48), (25150, 39), (25150, 39))

In [28]:
test_data_b.inputs.shape, test_data_b.pressures.shape, test_data_b.u_outs.shape

((25150, 39, 48), (25150, 39), (25150, 39))

In [29]:
NUMERICAL_INPUTS_CNT = len(num_cols)
N_FEATS = train_data.inputs.shape[2]
WINDOW_SIZE=train_data.inputs.shape[1]

In [30]:
# del train, test
# _ = gc.collect()

In [31]:
NUMERICAL_INPUTS_CNT

45

In [32]:
def custom_mean_absolute_error(y_true, y_pred, w_out):
    return K.mean(K.abs(y_pred - y_true))

def custom_metric(y_true, y_pred, w_out):
    return mae(y_true[w_out == 0], y_pred[w_out == 0])

def get_LSTM_model(Config):
    w_out = Input(shape=(WINDOW_SIZE, 1))
    targets = Input(shape=(WINDOW_SIZE, 1))
    inputs = Input(shape=(WINDOW_SIZE, N_FEATS))
    
    
    num_inputs = inputs[:, :, :NUMERICAL_INPUTS_CNT]
    cat_inputs = inputs[:, :, NUMERICAL_INPUTS_CNT:]

    initializer = tf.keras.initializers.glorot_uniform(seed=66)

    for i, c in enumerate(cat_cols):
        cat_inp = cat_inputs[:, :, i:i+1]
        embed = Embedding(input_dim=cat_cols_unq_dct[c]+2, output_dim=Config.CAT_DIM, embeddings_initializer=initializer)(cat_inp)
        reshaped = tf.reshape(embed, shape=(-1, embed.shape[1],  embed.shape[2] * embed.shape[3]))
        reshaped = SpatialDropout1D(0.1, seed=2)(reshaped)
        num_inputs = concatenate([reshaped, num_inputs], axis=2)
    
    
    x = num_inputs

    for i in range(Config.N_LSTM_LAYERS):
        x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(Config.LSTM_SIZE, return_sequences=True))(x)

    x = tf.keras.layers.Dense(Config.DENSE_SIZE, activation='relu')(x)
    out = tf.keras.layers.Dense(1)(x)
    
    model = tf.keras.Model(inputs=[inputs, targets, w_out], outputs=out)
    model.add_loss(custom_mean_absolute_error(targets, out, w_out))
    model.compile(optimizer=Adam(), loss=None)
    
    return model

In [33]:
class IntervalEvaluation(tf.keras.callbacks.Callback):
    def __init__(self, validation_data=(), interval=10, save_model=True, Config={}, fold=0, wandb_enabled=True):

        super(tf.keras.callbacks.Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val, self.w_out_val = validation_data
        self.best_score = np.inf
        self.save_model = save_model
        self.learning_rates = pd.Series(index=np.arange(Config.N_EPOCHS),
                                        data=np.linspace(Config.END_LR,
                                                         Config.START_LR,
                                                         num=Config.N_EPOCHS)[::-1]).to_dict()
        self.wandb_enabled = wandb_enabled
        

    def on_epoch_begin(self, epoch, logs={}):
        self.start_time = time()
        tf.keras.backend.set_value(self.model.optimizer.lr, self.learning_rates[epoch])
        self.curr_lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        
    def on_epoch_end(self, epoch, logs={}):
        keys = list(logs.keys())
        
        y_pred = self.model.predict([self.X_val, self.y_val, self.w_out_val], verbose=0, batch_size=64*32, use_multiprocessing=False)
        val_score = custom_metric(self.y_val.flatten(), y_pred.flatten(), self.w_out_val.flatten())
        
        if val_score < self.best_score:
            self.best_score = val_score
            if self.save_model:
                tf.keras.models.save_model(self.model, f'best_model', save_format="h5")

        total_time = round(time() - self.start_time, 2)
        if self.wandb_enabled:
            wandb.log({f'Fold {fold} epoch': epoch, f"Fold {fold} train_loss": logs['loss'], f"Fold {fold} val_score": val_score, f'Fold {fold} best_val_score': self.best_score, f'Fold {fold} total_time_in_seconds': total_time})
        
        if epoch % self.interval == 0:
            print(f"Epoch: {epoch:03d} curr_lr: {self.curr_lr:.1e} - train_loss: {logs['loss']:.03f} val_score: {val_score:.03f}  best_val_score: {self.best_score:.03f}  last_epoch t={total_time}s")

In [34]:
class Config:
    N_SPLITS = 10
    N_EPOCHS = 200
    BATCH_SIZE = 128
    CAT_DIM = 256
    START_LR = 5e-4
    END_LR = 1e-5
    LSTM_SIZE = 512
    DENSE_SIZE = 512
    N_LSTM_LAYERS = 3
    DROPOUT = 0.1
    SEQUENCE_LENGTH = 39
    LOG_NUM_COLS = False

Config_dct = {key:value for key, value in Config.__dict__.items() if not key.startswith('__') and not callable(key)}

In [ ]:
if WANDB:
    wandb.init(project="google-brain",
               name=f'nikhil_pressure_prediction_dl_{ver}',
               save_code=True,
               config=Config_dct)

In [ ]:
train_data.inputs.shape, test_data_a.inputs.shape

((75450, 39, 48), (25150, 39, 48))

In [ ]:
history_lst = []
oofs = np.zeros(train_data.pressures.shape)
preds_lst_a = []
preds_lst_b = []
kf = KFold(n_splits=Config.N_SPLITS, shuffle=True, random_state=101)

for fold, (trn_idx, val_idx) in enumerate(kf.split(train_data.pressures, train_data.pressures)):

        print('-'*15, '>', f'Fold {fold}', '<', '-'*15, '\n')
        X_trn, X_val = train_data.inputs[trn_idx], train_data.inputs[val_idx]
        


        y_trn, y_val = train_data.pressures[trn_idx], train_data.pressures[val_idx]
        wt_trn, wt_val = train_data.u_outs[trn_idx], train_data.u_outs[val_idx]

        if fold % 2 == 0:
            X_trn = np.concatenate((X_trn, test_data_a.inputs))
            y_trn = np.concatenate((y_trn, test_data_a.pressures))
            wt_trn = np.concatenate((wt_trn, test_data_a.u_outs))
        
        else:
            X_trn = np.concatenate((X_trn, test_data_b.inputs))
            y_trn = np.concatenate((y_trn, test_data_b.pressures))
            wt_trn = np.concatenate((wt_trn, test_data_b.u_outs))


        Config.STEPS_PER_EPOCH = len(X_trn)/Config.BATCH_SIZE
        model = get_LSTM_model(Config)

        print_val_results = IntervalEvaluation(
            validation_data=(X_val, y_val, wt_val),
            interval=1,
            Config=Config,
            fold=fold,
            wandb_enabled = WANDB
            )

        history = model.fit([X_trn, y_trn, wt_trn], None,
                    validation_data=([X_val, y_val, wt_val], None),
                    verbose=0,
                    epochs=Config.N_EPOCHS,
                    batch_size=Config.BATCH_SIZE,
                    callbacks=[print_val_results]
                    )

        history_lst.append(history)
        del model, X_trn, y_trn, wt_trn, print_val_results
        _ = gc.collect()

        best_model = load_model('best_model', custom_objects={'custom_mean_absolute_error': custom_mean_absolute_error})

        vp = best_model.predict([X_val, y_val, wt_val]).squeeze(-1)
        oofs[val_idx] = vp

        val_score = round(custom_metric(y_val.flatten(), vp.flatten(), wt_val.flatten()), 4)
        print(f'\nMAE val: {val_score}')

        if fold % 2 == 0:
            tp_b = best_model.predict([test_data_b.inputs, test_data_b.pressures, test_data_b.u_outs]).squeeze(-1)
            preds_lst_b.append(tp_b)

        else:
            tp_a = best_model.predict([test_data_a.inputs, test_data_a.pressures, test_data_a.u_outs]).squeeze(-1)
            preds_lst_a.append(tp_a)


        del X_val, y_val, wt_val, best_model
        _ = gc.collect()

oof_score = round(custom_metric(train_data.pressures.flatten(), oofs.flatten(), train_data.u_outs.flatten()), 4)
print(f'\nMAE OOF: {oof_score}')
if WANDB:
    wandb.finish()

In [ ]:
preds_a = np.median(np.array(preds_lst_a), axis=0)
preds_b = np.median(np.array(preds_lst_b), axis=0)
preds_a.shape, preds_b.shape

In [ ]:
preds_df_a = test_a[['id',]]
preds_df_b = test_b[['id']]

In [ ]:
preds_df_a['pressure'] = preds_a.flatten()
preds_df_b['pressure'] = preds_b.flatten()

In [ ]:
sample_preds_df = pd.concat([preds_df_a, preds_df_b]).reset_index(drop=True)
sample_preds_df

In [ ]:
sample_oof_df['pressure'] = oofs.flatten()

In [ ]:
print(submission.shape)
submission = pd.merge(submission[['id']], sample_preds_df, on='id', how='left')
submission['pressure'] = submission['pressure'].fillna(0)
print(submission.shape)

In [ ]:
print(oof_df.shape)
oof_df = pd.merge(oof_df[['id']], sample_oof_df, on='id', how='left')
oof_df['pressure'] = oof_df['pressure'].fillna(0)
print(oof_df.shape)

In [ ]:
mae(target[fltr_out], oof_df['pressure'][fltr_out])

In [ ]:
path = DATA_PATH + os.sep + f'dl_oof_{ver}.csv'
oof_df.to_csv(path, index=False)

In [ ]:
path = DATA_PATH + os.sep + f'dl_sub_{ver}.csv'
submission.to_csv(path, index=False)